<a href="https://colab.research.google.com/github/K-SAHASRA/GNN-modelling/blob/main/Gnn_modelling_art.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install pandas
!pip install scikit-learn




  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
from google.colab import files

uploaded = files.upload()


Saving 100-iterations data combined.csv to 100-iterations data combined.csv


In [4]:
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv


file_path = '/content/100-iterations data combined.csv'

data = pd.read_csv(file_path, low_memory=False)
print("Original data:")
print(data.head())

# Check data types there is a k and m somewhere
print("\nData types before conversion:")
print(data.dtypes)

# non-numeric conversion
data_cleaned = data.drop(columns=['Time'])

# Converting k and m
def custom_convert(series):
    series = series.str.replace(' K', '000')
    series = series.str.replace(' M', '000000')
    return pd.to_numeric(series, errors='coerce')

# cleaning
data_cleaned = data_cleaned.apply(lambda col: custom_convert(col) if col.dtype == 'object' else col)
print("Data after conversion:")
print(data_cleaned.head())
data_cleaned = data_cleaned.fillna(0)  #Filling NaNs with 0(few nans in dataset?)

# Normalize and scale values to between 0 and 1
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)
print("Data normalization successful.")

node_features = torch.tensor(normalized_data, dtype=torch.float)

# Create a fully connected graph(no proper defined relations so a fully connected)
num_nodes = node_features.shape[1]
adj_matrix = torch.ones((num_nodes, num_nodes)) - torch.eye(num_nodes)
edge_index = dense_to_sparse(adj_matrix)[0]

# Prepare time-series data
time_window = 10  # Number of time steps(rows) to consider
node_features_time_series = []
target_time_series = []

target_index = list(data_cleaned.columns).index('Average Response Time')

# Loop through the data
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][target_index])

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Create graph data object(why? its the object we feed to the monster model)
graph_data = Data(x=node_features_time_series, edge_index=edge_index, y=target_time_series)

Original data:
             Time  Avg Host Node1 CPU Usage  Host Node1 CPU 0 Usage  \
0  6/28/2024 0:00                      24.1                    19.4   
1  6/28/2024 0:00                      23.4                    25.8   
2  6/28/2024 0:00                      23.1                    21.4   
3  6/28/2024 0:00                      23.1                    24.2   
4  6/28/2024 0:00                      23.2                    20.0   

   Host Node1 CPU 1 Usage  Avg Host Node2 CPU Usage  Host Node2 CPU 0 Usage  \
0                    28.8                      20.3                    21.4   
1                    21.0                      19.5                    17.6   
2                    24.8                      19.4                    20.6   
3                    22.0                      19.3                    22.6   
4                    26.4                      20.4                    23.4   

   Host Node2 CPU 1 Usage  Avg Host Node3 CPU Usage  Host Node3 CPU 0 Usage  \
0   

In [17]:

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)

node_features = torch.tensor(normalized_data, dtype=torch.float)

# scale the target variable (Average Response Time)
target_scaler = MinMaxScaler()
target_column = data_cleaned[['Average Response Time']]
target_scaler.fit(target_column)

# Convert to torch tensor
node_features = torch.tensor(normalized_data, dtype=torch.float)

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

target_index = list(data_cleaned.columns).index('Average Response Time')


for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][target_index])

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Now, create the edge_index for each time step
num_nodes = node_features_time_series.size(1)
# Create a fully connected graph for each time step
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

# Replicate edge_index for each row in the sequence
edge_index = edge_index.repeat(1, node_features_time_series.size(0))

# Flatten node features for the graph(why? figure this )
graph_data = Data(x=node_features_time_series.view(-1, node_features_time_series.size(2)), edge_index=edge_index, y=target_time_series)

# Define the GNN model(2 layers well technically 3 layers)
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        # Graph Convolutional Layer
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # Attention Layer
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        # Fully connected layer for output
        self.fc1 = nn.Linear(hidden_channels * 4, out_channels)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Graph Convolution
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        # Attention Mechanism
        x = self.att1(x, edge_index)
        x = torch.relu(x)
        # Output Layer
        x = self.fc1(x)
        # Average across nodes to get a single prediction per time step
        x = torch.mean(x.view(-1, num_nodes, x.size(-1)), dim=1)
        return x

model = GNNModel(in_channels=node_features_time_series.size(2), hidden_channels=32, out_channels=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training the model
def train(model, data, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, graph_data)

Epoch 1, Loss: 0.044345758855342865
Epoch 2, Loss: 0.02961917780339718
Epoch 3, Loss: 0.019507762044668198
Epoch 4, Loss: 0.013611536473035812
Epoch 5, Loss: 0.011186876334249973
Epoch 6, Loss: 0.011347515508532524
Epoch 7, Loss: 0.012936381623148918
Epoch 8, Loss: 0.014634435065090656
Epoch 9, Loss: 0.015542705543339252
Epoch 10, Loss: 0.015429528430104256
Epoch 11, Loss: 0.01452124584466219
Epoch 12, Loss: 0.01320298295468092
Epoch 13, Loss: 0.011840746738016605
Epoch 14, Loss: 0.010704978369176388
Epoch 15, Loss: 0.009948878549039364
Epoch 16, Loss: 0.009610463865101337
Epoch 17, Loss: 0.009631585329771042
Epoch 18, Loss: 0.009888950735330582
Epoch 19, Loss: 0.010231247171759605
Epoch 20, Loss: 0.010522444732487202
Epoch 21, Loss: 0.01067386381328106
Epoch 22, Loss: 0.010653828270733356
Epoch 23, Loss: 0.010480313561856747
Epoch 24, Loss: 0.010203892365098
Epoch 25, Loss: 0.0098891481757164
Epoch 26, Loss: 0.00959704164415598
Epoch 27, Loss: 0.009374936111271381
Epoch 28, Loss: 0.00

In [21]:
#calling the monster
model.eval()
with torch.no_grad():
    predictions = model(graph_data)

# Print predictions
predictions = predictions.view(-1).tolist()
for i, pred in enumerate(predictions):
    print(f'Time step {i}: Predicted Average Response Time = {pred:.4f}')




Streaming output truncated to the last 5000 lines.
Time step 12882: Predicted Average Response Time = 0.0681
Time step 12883: Predicted Average Response Time = 0.0664
Time step 12884: Predicted Average Response Time = 0.0647
Time step 12885: Predicted Average Response Time = 0.0839
Time step 12886: Predicted Average Response Time = 0.0939
Time step 12887: Predicted Average Response Time = 0.0937
Time step 12888: Predicted Average Response Time = 0.0936
Time step 12889: Predicted Average Response Time = 0.0951
Time step 12890: Predicted Average Response Time = 0.0949
Time step 12891: Predicted Average Response Time = 0.0952
Time step 12892: Predicted Average Response Time = 0.0943
Time step 12893: Predicted Average Response Time = 0.0958
Time step 12894: Predicted Average Response Time = 0.0954
Time step 12895: Predicted Average Response Time = 0.0752
Time step 12896: Predicted Average Response Time = 0.0644
Time step 12897: Predicted Average Response Time = 0.0642
Time step 12898: Pred

In [22]:
predictions_tensor = torch.tensor(predictions)

# Reshape the tensor and move it to the CPU if needed
predictions_tensor = predictions_tensor.view(-1, 1).cpu().numpy()  # Convert to numpy for inverse transform(remember u saved this transform earlier)


denormalized_predictions = target_scaler.inverse_transform(predictions_tensor)
# Apply ReLU to set any negative values to zero(why? negative values happens because our dataset have some really small values close to zero
#and during the loss function they are going negative anyways since negative response time does not mean anything
#so applying  relu function to set the values less than 0 to 0)
denormalized_predictions = np.maximum(0, denormalized_predictions)
print("Denormalized Predictions:")
# Print denormalized predictions
for i, pred in enumerate(denormalized_predictions):
    print(f'Time step {i}: Predicted Average Response Time = {pred[0]:.4f}')

Streaming output truncated to the last 5000 lines.
Time step 12882: Predicted Average Response Time = 19.7506
Time step 12883: Predicted Average Response Time = 19.2460
Time step 12884: Predicted Average Response Time = 18.7546
Time step 12885: Predicted Average Response Time = 24.3363
Time step 12886: Predicted Average Response Time = 27.2279
Time step 12887: Predicted Average Response Time = 27.1860
Time step 12888: Predicted Average Response Time = 27.1528
Time step 12889: Predicted Average Response Time = 27.5677
Time step 12890: Predicted Average Response Time = 27.5216
Time step 12891: Predicted Average Response Time = 27.6042
Time step 12892: Predicted Average Response Time = 27.3537
Time step 12893: Predicted Average Response Time = 27.7696
Time step 12894: Predicted Average Response Time = 27.6587
Time step 12895: Predicted Average Response Time = 21.8044
Time step 12896: Predicted Average Response Time = 18.6649
Time step 12897: Predicted Average Response Time = 18.6052
Time 